In [6]:
import pandas as pd
import urllib.parse
import requests
import datetime
import os
import json
from bs4 import BeautifulSoup
import sqlite3

In [2]:
def remove_spaces(input_string):
    # Use str.replace() to replace spaces with an empty string
    result_string = input_string.replace(" ", "")
    return result_string

def create_search_link(query: str, lang, geo_coordinates, zoom):
    # Check for invalid combination of geo_coordinates and zoom
    if geo_coordinates is None and zoom is not None:
        raise ValueError("geo_coordinates must be provided along with zoom")

    # URL encoding the query
    endpoint = urllib.parse.quote_plus(query)

    # Basic parameters
    params = {
            'authuser': '0',
            'hl': lang,
            'entry': 'ttu',

    } if lang is not None else {
            'authuser': '0',
            'entry': 'ttu',
          
    }


    # Constructing the geo-coordinates string
    geo_str = ""
    if geo_coordinates is not None:
        geo_coordinates = remove_spaces(geo_coordinates)
        if zoom is not None:
            geo_str = f'/@{geo_coordinates},{zoom}z'
        else:
            geo_str = f'/@{geo_coordinates}'

    # Constructing the final URL
    url = f'https://www.google.com/maps/search/{endpoint}'
    if geo_str:
        url += geo_str
    url += f'?{urllib.parse.urlencode(params)}'

    return url

In [3]:
jenis = 'restaurant'
kecamatan = 'gambir'

In [4]:
df_pos = pd.read_csv('../scrape_kode_pos_indonesia/output/kode_pos.csv')
df_pos = df_pos.fillna('-')
df_cari = df_pos[df_pos['KECAMATAN'] == kecamatan.upper()]
df_cari

,PROPINSI,KOTA,KECAMATAN,KELURAHAN,KODE POS,UPDATE
19525,DKI JAKARTA,KOTA ADM. JAKARTA PUSAT,GAMBIR,GAMBIR,10110.0,2024-04-04 11:59:02.125637
19526,DKI JAKARTA,KOTA ADM. JAKARTA PUSAT,GAMBIR,CIDENG,10150.0,2024-04-04 11:59:02.132541
19527,DKI JAKARTA,KOTA ADM. JAKARTA PUSAT,GAMBIR,PETOJO UTARA,10130.0,2024-04-04 11:59:02.132541
19528,DKI JAKARTA,KOTA ADM. JAKARTA PUSAT,GAMBIR,PETOJO SELATAN,10160.0,2024-04-04 11:59:02.142171
19529,DKI JAKARTA,KOTA ADM. JAKARTA PUSAT,GAMBIR,KEBON KELAPA,10120.0,2024-04-04 11:59:02.142714
19530,DKI JAKARTA,KOTA ADM. JAKARTA PUSAT,GAMBIR,DURI PULO,10140.0,2024-04-04 11:59:02.147791


In [ ]:
for i in range(0, len(df_cari)):

    cari = f'{jenis} in {df_cari.iloc[i][3]} {df_cari.iloc[i][2]} {df_cari.iloc[i][1]} {df_cari.iloc[i][0]}'
    url_cari = create_search_link(cari, None, '', 18)
    response = requests.get(url_cari)

    if response.status_code == 200:
        data_cari = response.text
        soup_cari = BeautifulSoup(data_cari, 'html.parser')
        scripts = soup_cari.find_all('script')

        for script in scripts:
            if 'window.APP_INITIALIZATION_STATE' in str(script):
                data = str(script).split('=',3)[3]
                data2 = data.rsplit(';',10)[0]
                json_data = json.loads(data2)
                usaha = json_data[3][2][5:]
                json_usaha = json.loads(usaha)

                for a in range(1,20):
                    try:
                        nama = json_usaha[0][1][a][14][11]
                        koordinat = ', '.join(list(map(str, json_usaha[0][1][a][14][9][-2:])))
                        alamat = ', '.join(json_usaha[0][1][a][14][2])

                        df_data = pd.concat([df_data, pd.Series({'NAMA': nama,
                                                                'KOORDINAT': koordinat,
                                                                'ALAMAT': alamat,
                                                                'KOTA': df_cari.iloc[i][1],
                                                                'KECAMATAN': df_cari.iloc[i][2],
                                                                'KELURAHAN': df_cari.iloc[i][3]}).to_frame().T], ignore_index=True)
                    except Exception:
                        print(f'Gagal query kelurahan {df_cari.iloc[i][3]} baris {a}')
                    
        print(f'Kode pos {df_cari.iloc[i][4]} Kecamatan {df_cari.iloc[i][3]} Kota {df_cari.iloc[i][2]} telah disimpan')

    else:
        print(f'Gagal query. Status code: {response.status_code} untuk kelurahan {df_cari.iloc[i][3]} kecamatan {df_cari.iloc[i][2]}')

df_data.to_csv(f'./output/{jenis}_{df_cari.iloc[i][2]}_{df_cari.iloc[i][1]}.csv', index=False)

In [5]:
df_data = pd.DataFrame(columns=['NAMA', 'KOORDINAT', 'ALAMAT', 'KOTA', 'KECAMATAN', 'KELURAHAN'])

for i in range(0, len(df_cari)):

    cari = f'{jenis} in {df_cari.iloc[i][3]} {df_cari.iloc[i][2]} {df_cari.iloc[i][1]} {df_cari.iloc[i][0]}'
    url_cari = create_search_link(cari, None, '', 18)
    response = requests.get(url_cari)

    if response.status_code == 200:
        data_cari = response.text
        soup_cari = BeautifulSoup(data_cari, 'html.parser')
        scripts = soup_cari.find_all('script')

        for script in scripts:
            if 'window.APP_INITIALIZATION_STATE' in str(script):
                data = str(script).split('=',3)[3]
                data2 = data.rsplit(';',10)[0]
                json_data = json.loads(data2)
                usaha = json_data[3][2][5:]
                json_usaha = json.loads(usaha)

                for a in range(1,20):
                    try:
                        nama = json_usaha[0][1][a][14][11]
                        koordinat = ', '.join(list(map(str, json_usaha[0][1][a][14][9][-2:])))
                        alamat = ', '.join(json_usaha[0][1][a][14][2])

                        df_data = pd.concat([df_data, pd.Series({'NAMA': nama,
                                                                'KOORDINAT': koordinat,
                                                                'ALAMAT': alamat,
                                                                'KOTA': df_cari.iloc[i][1],
                                                                'KECAMATAN': df_cari.iloc[i][2],
                                                                'KELURAHAN': df_cari.iloc[i][3]}).to_frame().T], ignore_index=True)
                    except Exception:
                        print(f'Gagal query kelurahan {df_cari.iloc[i][3]} baris {a}')
                    
        print(f'Kode pos {df_cari.iloc[i][4]} Kecamatan {df_cari.iloc[i][3]} Kota {df_cari.iloc[i][2]} telah disimpan')

    else:
        print(f'Gagal query. Status code: {response.status_code} untuk kelurahan {df_cari.iloc[i][3]} kecamatan {df_cari.iloc[i][2]}')

df_data.to_csv(f'./output/{jenis}_{df_cari.iloc[i][2]}_{df_cari.iloc[i][1]}.csv', index=False)

C:\Users\van-g\AppData\Local\Temp\ipykernel_1644\1271263666.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cari = f'{jenis} in {df_cari.iloc[i][3]} {df_cari.iloc[i][2]} {df_cari.iloc[i][1]} {df_cari.iloc[i][0]}'
C:\Users\van-g\AppData\Local\Temp\ipykernel_1644\1271263666.py:31: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'KOTA': df_cari.iloc[i][1],
C:\Users\van-g\AppData\Local\Temp\ipykernel_1644\1271263666.py:32: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value 

Kode pos 10110.0 Kecamatan GAMBIR Kota GAMBIR telah disimpan


C:\Users\van-g\AppData\Local\Temp\ipykernel_1644\1271263666.py:31: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'KOTA': df_cari.iloc[i][1],
C:\Users\van-g\AppData\Local\Temp\ipykernel_1644\1271263666.py:32: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'KECAMATAN': df_cari.iloc[i][2],
C:\Users\van-g\AppData\Local\Temp\ipykernel_1644\1271263666.py:33: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'KELURAHAN': df_cari.iloc[i][3]}).to

Gagal query kelurahan CIDENG baris 17
Gagal query kelurahan CIDENG baris 18
Gagal query kelurahan CIDENG baris 19
Kode pos 10150.0 Kecamatan CIDENG Kota GAMBIR telah disimpan


C:\Users\van-g\AppData\Local\Temp\ipykernel_1644\1271263666.py:31: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'KOTA': df_cari.iloc[i][1],
C:\Users\van-g\AppData\Local\Temp\ipykernel_1644\1271263666.py:32: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'KECAMATAN': df_cari.iloc[i][2],
C:\Users\van-g\AppData\Local\Temp\ipykernel_1644\1271263666.py:33: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'KELURAHAN': df_cari.iloc[i][3]}).to

...

C:\Users\van-g\AppData\Local\Temp\ipykernel_1644\1271263666.py:31: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'KOTA': df_cari.iloc[i][1],
C:\Users\van-g\AppData\Local\Temp\ipykernel_1644\1271263666.py:32: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'KECAMATAN': df_cari.iloc[i][2],
C:\Users\van-g\AppData\Local\Temp\ipykernel_1644\1271263666.py:33: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'KELURAHAN': df_cari.iloc[i][3]}).to

Kode pos 10160.0 Kecamatan PETOJO SELATAN Kota GAMBIR telah disimpan


C:\Users\van-g\AppData\Local\Temp\ipykernel_1644\1271263666.py:31: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'KOTA': df_cari.iloc[i][1],
C:\Users\van-g\AppData\Local\Temp\ipykernel_1644\1271263666.py:32: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'KECAMATAN': df_cari.iloc[i][2],
C:\Users\van-g\AppData\Local\Temp\ipykernel_1644\1271263666.py:33: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'KELURAHAN': df_cari.iloc[i][3]}).to

Kode pos 10120.0 Kecamatan KEBON KELAPA Kota GAMBIR telah disimpan
Gagal query kelurahan DURI PULO baris 1
Gagal query kelurahan DURI PULO baris 2
Gagal query kelurahan DURI PULO baris 3
Gagal query kelurahan DURI PULO baris 4
Gagal query kelurahan DURI PULO baris 5
Gagal query kelurahan DURI PULO baris 6
Gagal query kelurahan DURI PULO baris 7
Gagal query kelurahan DURI PULO baris 8
Gagal query kelurahan DURI PULO baris 9
Gagal query kelurahan DURI PULO baris 10
Gagal query kelurahan DURI PULO baris 11
Gagal query kelurahan DURI PULO baris 12
Gagal query kelurahan DURI PULO baris 13
Gagal query kelurahan DURI PULO baris 14
Gagal query kelurahan DURI PULO baris 15
Gagal query kelurahan DURI PULO baris 16
Gagal query kelurahan DURI PULO baris 17
Gagal query kelurahan DURI PULO baris 18
Gagal query kelurahan DURI PULO baris 19
Kode pos 10140.0 Kecamatan DURI PULO Kota GAMBIR telah disimpan


C:\Users\van-g\AppData\Local\Temp\ipykernel_1644\1271263666.py:35: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f'Gagal query kelurahan {df_cari.iloc[i][3]} baris {a}')
C:\Users\van-g\AppData\Local\Temp\ipykernel_1644\1271263666.py:42: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_data.to_csv(f'./output/{jenis}_{df_cari.iloc[i][2]}_{df_cari.iloc[i][1]}.csv', index=False)
